In [ ]:
%load_ext autoreload
%autoreload 2
    
import os, getpass

from dotenv import load_dotenv

load_dotenv()

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("GOOGLE_API_KEY")
_set_env("TAVILY_API_KEY")

In [ ]:
import warnings
import urllib3
from requests import sessions

# Disable SSL warnings when verification is disabled
warnings.filterwarnings('ignore', message='Unverified HTTPS request')
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Monkey patch requests to disable SSL verification for corporate proxy compatibility
original_request = sessions.Session.request

def patched_request(self, method, url, *args, **kwargs):
    kwargs['verify'] = False
    return original_request(self, method, url, *args, **kwargs)

sessions.Session.request = patched_request


In [ ]:
def print_travel_destinations(traveldestinations):
    if travel_recommendations:
        for t in traveldestinations:
            print(f"Country: {t.country}")
            print(f"City: {t.city}")
            print(f"Start Date: {t.startDate}")
            print(f"End Date: {t.endDate}")
            print(f"Popular Places: {t.popular_places}")
            print("Recommended Stars: ", end='')
            print("★" * t.recommended_star)
            print(f"Budget: {t.estimated_budget}")
            print("-" * 50)

In [ ]:
from studio.travel_assistant import  graph

In [ ]:
from IPython.display import Image, display, Markdown

In [ ]:
display(Image(graph.get_graph(xray=1).draw_mermaid_png()))

In [ ]:
# Input
max_destinations = 3 
preferences = ["mountain", "beaches"] 
budget = 1000
thread = {"configurable": {"thread_id": "5"}}

# Run the graph until the first interruption
for event in graph.stream({
        "preferences": preferences,
        "max_destinations": max_destinations
    }, thread, stream_mode="values"):
    # Review
    travel_recommendations = event.get('travel_recommendations', [])
    print_travel_destinations(travel_recommendations) 


In [ ]:
state = graph.get_state(thread)
state.next 

In [ ]:
graph.update_state(thread, {"human_feedback": 
                            "Add some places from Canada"}, as_node="human_feedback")

In [ ]:
state = graph.get_state(thread)
state.next

In [ ]:
for event in graph.stream(None, thread, stream_mode="values"):
    # Review
    travel_recommendations = event.get('travel_recommendations', [])
    print_travel_destinations(travel_recommendations) 

In [ ]:
graph.update_state(thread, {"human_feedback": 
                            "approve"}, as_node="human_feedback")

In [ ]:
for event in graph.stream(None, thread, stream_mode="updates"):
    print("--Node--")
    node_name = next(iter(event.keys()))
    print(node_name)

In [ ]:
final_state = graph.get_state(thread)
travel_recommendations = final_state.values.get('travel_recommendations', [])
print_travel_destinations(travel_recommendations) 

In [ ]:
travel_itineraries = final_state.values.get('travel_itineraries', [])
for t in travel_itineraries:
 
    display(Markdown(t))
    print("-" * 50)

In [ ]:
display(Markdown(final_state.values.get('best_travel_destination', "")))